# Выбор локации для скважины

Допустим, мы работаем в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Нам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Построим модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируем возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error
from numpy.random import RandomState
state = RandomState(12345)

In [2]:
def rmse_mean(data, count=0):
    x_train, x_valid, y_train, y_valid = train_test_split(data.drop('product', axis=1), data['product'], random_state=12345, test_size=.25)
    lr = LinearRegression()
    lr.fit(x_train, y_train)
    lr_predict = lr.predict(x_valid)
    rmse = mean_absolute_error(y_valid, lr_predict)**(1/2)
    mean_value = lr_predict.mean()
    print(f'Для региона {count} RMSE={rmse} и средний запас предсказанного сырья={mean_value}')
    return lr, lr_predict

In [3]:
#расчёт прибыли

def profit(data, count=200, budget=10000000000, one_barrel=450000):
    data = data.sort_values(by='predict', ascending=False)
    selected = data['true'][:count] 
    gross_profit = selected.sum() * one_barrel - budget # валовая прибыль
    return gross_profit
    

In [4]:
def bootstrap(data, state=RandomState(12345)):
    
    x_train, x_valid, y_train, y_valid = train_test_split(data.drop('product', axis=1), data['product'], random_state=12345, test_size=.25)
    lr = LinearRegression()
    lr.fit(x_train, y_train)
    lr_predict = pd.DataFrame(lr.predict(x_valid), columns=['predict']).reset_index(drop=True)
    y_valid = pd.DataFrame(y_valid).rename(columns={'product':'true'}).reset_index(drop=True)
    
    values = []
    data_for_sample = pd.concat([y_valid, lr_predict], axis=1)#.sort_values(by='predict', ascending=False)
    
    for i in range(1000):
    
        data_subsample = data_for_sample.sample(n=500, replace=True, random_state=state)
        

        values.append(profit(data_subsample))

    values = pd.Series(values)
    lower = values.quantile(0.025) # < напишите код здесь>
    upper = values.quantile(0.975)
    mean = values.mean()
    print("Средняя прибыль:", mean)
    print("2.5%-квантиль:", lower)
    print("97.5-квантиль:", upper)
    
    values = pd.DataFrame(values, columns=['profit'])
    minus_profit = values.loc[values['profit']<0].count()[0]
    procent = (minus_profit / values.shape[0])*100
    print(f'Риск убытков: {round(procent, 2)}%')

In [5]:
geo_data1 = pd.read_csv('/datasets/geo_data_0.csv')
geo_data2 = pd.read_csv('/datasets/geo_data_1.csv')
geo_data3 = pd.read_csv('/datasets/geo_data_2.csv')

In [6]:
geo_data1.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [7]:
geo_data2.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [8]:
geo_data3.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [9]:
geo_data1 = geo_data1.iloc[:,1:]
geo_data2 = geo_data2.iloc[:,1:]
geo_data3 = geo_data3.iloc[:,1:]

#### Откинем колонку id, тк она не несёт смысловой нагрузки для машинного обучения.

In [10]:
geo_data2.head()

,f0,f1,f2,product
0,-15.001348,-8.276000,-0.005876,3.179103
1,14.272088,-3.475083,0.999183,26.953261
2,6.263187,-5.948386,5.001160,134.766305
3,-13.081196,-11.506057,4.999415,137.945408
4,12.702195,-8.147433,5.004363,134.766305


In [11]:
geo_data1.isna().sum()

f0         0
f1         0
f2         0
product    0
dtype: int64

In [12]:
geo_data2.isna().sum()

f0         0
f1         0
f2         0
product    0
dtype: int64

In [13]:
geo_data3.isna().sum()

f0         0
f1         0
f2         0
product    0
dtype: int64

In [14]:
geo_data1.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


#### Вывод: данные чисты и не требуют дальнейшей обработки.

## Обучение и проверка модели

### 1 регион:

In [15]:
lr1, pred1 = rmse_mean(geo_data1, count=1)

Для региона 1 RMSE=5.560539612047675 и средний запас предсказанного сырья=92.59256778438035


### 2 регион:

In [16]:
lr2, pred2 = rmse_mean(geo_data2, count=2)

Для региона 2 RMSE=0.8478008281503834 и средний запас предсказанного сырья=68.728546895446


### 3 регион:

In [17]:
lr3, pred3 = rmse_mean(geo_data3, count=3)

Для региона 3 RMSE=5.726486890361473 и средний запас предсказанного сырья=94.96504596800489


#### Мне кажется, что для первого региона значение rmse относительно среднего предсказанного немного великовато, желательно получить 7 и ниже, однако если так подумать, то 15, в теории, тоже не особо плохо.

#### Для второго региона RMSE - идеал.

#### Для 3-его региона вывод аналогичен первому.

#### Вывод: модель показала адекватный rmse и адекватное среднее предсказания.

## Подготовка к расчёту прибыли

In [18]:
budget = 10000000000
all_counts = 500
max_counts = 200
one_barrel = 450 #прибыль с одной тонны (барреля) продукта (450 рублей)
counts_barrel = 1000

In [19]:
profit_volume = budget / (one_barrel * counts_barrel * max_counts)
profit_volume

111.11111111111111

In [20]:
pred1.mean()

92.59256778438035

In [21]:
pred2.mean()

68.728546895446

In [22]:
pred3.mean()

94.96504596800489

#### Вывод: прогноз не внушительный... Так как если верить среднему, то в каждом регионе среднее меньше, чем нужно нам.

## Расчёт прибыли и рисков 

In [23]:
bootstrap(geo_data1)

Средняя прибыль: 396164984.8023711
2.5%-квантиль: -111215545.89049526
97.5-квантиль: 909766941.5534226
Риск убытков: 6.9%


In [24]:
bootstrap(geo_data2)

Средняя прибыль: 461155817.2772397
2.5%-квантиль: 78050810.7517417
97.5-квантиль: 862952060.2637234
Риск убытков: 0.7%


In [25]:
bootstrap(geo_data3)

Средняя прибыль: 392950475.17060447
2.5%-квантиль: -112227625.37857565
97.5-квантиль: 934562914.5511636
Риск убытков: 6.5%


#### Вывод: финалистом стал 2-ий регион.

#### Вывод: с одной стороны 1 и 3 регионы вроде бы как богаче, но с другой там риск убытков выше, поэтому если брать риск как главный фактор выбора, то на основании Бутстрепа могу выдвинуть 2 регион.